# Alaska PPE Forecasting
## Editable View
This document provides direct access to the underlying data informing the projections. In addition to using sliders to adjust stockpile days, you can edit burn rate assumptions and rerun the model on the spot.

We provide detailed annotations of how to adjust model parameters for readers that might not have experience in programming or the Python programming language. However, if you do have a programming background, we expose our source code here for even more interactivity and editability. Simply use the button below to toggle on/off the raw code.

This is an alpha version of this view, so there are a couple extra steps to get started:

1. In the menu above, click on the `Kernel` button. A drop-down menu will appear; click `Restart and Run All`.
2. This will trigger a prompt asking you if you want to restart the kernel and re-run the whole notebook. Click the red button, `Restart and Run All Cells`. If errors begin to occur at any point, repeat this step of `Restart and Run All Cells`.

At this point, setup is complete.

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Editable Burn Rates
Click in the table below to adjust burn rate assumptions for any resource. Once your edits are done, hit `Enter`, and then click the `Run Model` button to adjust projections for all models below.

In [2]:
import os
import pathlib
import restart

# need this ugly variable for now to directly access config
PATH = pathlib.Path(restart.__file__).parent.absolute()

In [3]:
import sys
import pandas as pd
import numpy as np
from restart import RestartModel
from restart import Data
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells
from restart import util
from IPython.display import Javascript, display
import ipywidgets as widgets
import bqplot
from bqplot import pyplot as plt
import qgrid
import pathlib

qgrid.set_grid_option('forceFitColumns', False)
PATH = pathlib.Path(restart.__file__).parent.absolute()

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, \
                        IPython.notebook.ncells())'))
    
def create_run_button(description):
    button = widgets.Button(description=description)
    button.on_click(run_all)
    return button

restart = RestartModel(population='oes', state='Alaska')
model = restart.model

burn_df = model.demand.demand_per_unit_map_dn_um.df
burn_df.index.name = "Population"

burn_rates = qgrid.show_grid(burn_df)

In [4]:
display(create_run_button("Run Model"))
widgets.VBox([burn_rates])

Button(description='Run Model', style=ButtonStyle())

In [5]:
# get updated burn rates
new_arr = burn_rates.get_changed_df().to_numpy()
model.demand.adjust_burn(new_arr)

# charting 
burn_chart = util.generate_group_bar_legend(model.demand.demand_per_unit_map_dn_um.df)
widgets.VBox([burn_chart])

# General Population Forecast

In [6]:
import sys
import pandas as pd
import numpy as np
from restart import RestartModel
from restart import Data
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells
import ipywidgets as widgets
import bqplot
from bqplot import pyplot as plt

model.inventory.set_average_orders_per_period(model.demand.demand_by_popsum1_total_rp1n_tc)

slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = util.generate_group_bar_legend(df_chart, scientific_notation=True)
    display(sheet)
    display(chart)
    
def set_stock(backstop):
    backstop = [backstop]
    model.inventory.order(model.inventory.inv_by_popsum1_total_rp1n_tc)
    model.inventory.set_min_in_periods(backstop)
    display_stock(model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
wa_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
pop = format_population(to_sheet(model.population.population_pP_tr.df))
    
out = widgets.interactive_output(dashboard, {'backstop': slider})
widgets.VBox([slider, out])

# Healthcare Workers Forecast

In [7]:
def dashboard_h(backstop):
    set_stock_h(backstop)
    
def set_stock_h(backstop):
    backstop = [backstop]
    model_health.inventory.order(model_health.inventory.inv_by_popsum1_total_rp1n_tc)
    model_health.inventory.set_min_in_periods(backstop)
    display_stock(model_health.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
restart_health = RestartModel(population='oes', state='Alaska', subpop='healthcare')
model_health = restart_health.model
pop_health = format_population(to_sheet(model_health.population.population_pP_tr.df))

# get updated burn rates
model_health.demand.adjust_burn(new_arr)

model_health.inventory.set_average_orders_per_period(model_health.demand.demand_by_popsum1_total_rp1n_tc)

slider_health = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)
out_health = widgets.interactive_output(dashboard_h, {'backstop': slider_health})

widgets.VBox([slider_health, out_health])

# First Responders Forecast

In [8]:
def dashboard_fr(backstop):
    set_stock_fr(backstop)
    
def set_stock_fr(backstop):
    backstop = [backstop]
    model_fr.inventory.order(model_fr.inventory.inv_by_popsum1_total_rp1n_tc)
    model_fr.inventory.set_min_in_periods(backstop)
    display_stock(model_fr.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
restart_fr = RestartModel(population='oes', state='Alaska', subpop='wa_tier2_opt2')
model_fr = restart_fr.model
pop_fr = format_population(to_sheet(model_fr.population.population_pP_tr.df))

# get updated burn rates
model_fr.demand.adjust_burn(new_arr)

model_fr.inventory.set_average_orders_per_period(model_fr.demand.demand_by_popsum1_total_rp1n_tc)

slider_fr = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)
out_fr = widgets.interactive_output(dashboard_fr, {'backstop': slider_fr})

widgets.VBox([slider_fr, out_fr])